# PowMon app.py

a data accoution system that collects power consumption readings from arduinos using serial. 
the system works in a multitreaded style and can connect to muliple arduinos at the same time

## Import all dependencies and third party libraries

In [ ]:
import yaml
import serial
import pathlib
from cryptography.fernet import Fernet
import time
import threading
import requests
import os
from threading import Lock

## Global Vars
<b>OKBLUE,OKGREEN,OKCYAN,WARNING,FAIL</b> :  a number of global vars that are mainly used to style the color of the output <br>
<b>lock</b>: a lock var that is used to syncronize the write to file functionalty

In [ ]:
OKBLUE  = '\033[94m'
OKGREEN = '\033[92m'
OKCYAN = '\033[96m'
WARNING = '\033[93m'
FAIL = '\033[91m'
lock = Lock()

# Functions

### update_conf : 
a function to update the configuration file with updated configurations if they were changed 

In [ ]:
def update_conf():
    settings['EHOST'] = ""
    with open(settings['PROJECT_ROOT'] + "/settings.yaml", "w") as _file:
        yaml.dump(settings, _file)

### write_file
#### Parameters
<b>String - payload: </b> data to write to file
#### Description
a function that writes the passed string to the data file. this function checks if the current data file has reached the size limite then either creates a new file and calls the update_conf function or just saves the data 

In [ ]:
def write_file(payload):
    try:
        path = settings['LOCAL_DATA_ROOT'] + settings['LOCAL_DATA_FILE']
        fsize = os.path.getsize(path)
        if(fsize >= settings['FILE_SIZE_LIMIT']):
            settings['LOCAL_DATA_FILE'] = "/" + str(time.time()).replace('.' , '') + "-data.csv"
            path = settings['LOCAL_DATA_ROOT'] + settings['LOCAL_DATA_FILE']
            update_conf()
        with open( path, "a+") as file_object:
            file_object.write(payload)
        return True
    except Exception as e:
        print(FAIL + '[ERROR]' + e)
        return False

### local_save
#### Parameters
<b>String - data: </b> data to write to file
#### Description
a function that checks if SYNC_WRITE is enabled or not and saves the data accordingly

In [ ]:
def local_save(data):
    if(settings['SYNC_WRITE'] == True):
        with lock:
            return write_file(data)
    else:
        return write_file(data)

### log
#### Parameters
<b>String - sensor_id: </b> ID of the sensor that collected the data <br>
<b>String - payload: </b> collected data <br>
<b>String - rstatus: </b> Remote save status <br>
<b>String - sstatus: </b> Local save status <br>
#### Description
a function that prints to the console for user to judge and monitor the behavior of the system. the function has multiple log levels where level 0 will not print anything, level 1 will print sensor_id, rstatus, sstatus and level 2 will print the collected data (payload) and level 3 will print everything sensor_id, rstatus, sstatus, payload

In [ ]:
def log(sensor_id, payload, rstatus, sstatus):
    payload = payload.rstrip("\n")
    if(settings['LOG_LEVEL'] == 1):
        print(OKBLUE + "[DEBUG]\t" , sensor_id , "\t", rstatus, "\t" , sstatus)
    elif(settings['LOG_LEVEL'] == 2):
        print(OKBLUE +"[DEBUG]\t" ,payload)
    elif(settings['LOG_LEVEL'] == 3):
        print(OKBLUE +"[DEBUG]\t" , sensor_id , "\t", rstatus, "\t" , sstatus, "\t" ,payload)

### remote_save
#### Parameters
<b>String - payload: </b> data to be poseted to MDX Servers<br>
#### Description
a function that posts the collected data to MDX Servers to be stored as a backup. this can be disabled by the user through the config file

In [ ]:
def remote_save(payload):
    try: 
        r = requests.post(settings['EHOST'], data=payload, timeout=1)
        return r.status_code
    except Exception: 
        return 500 

### flush
#### Parameters
<b>Object - arduino: </b> the serial object to be flushed<br>
#### Description
a function that flushes and cleans the specified serial port to remove any unwanted data 

In [ ]:
def flush(arduino):
    arduino.write(str.encode('r \n'))
    arduino.readline()[:-2]
    time.sleep(0.001)
    arduino.write(str.encode('r \n'))
    arduino.readline()[:-2]

### read_serial
#### Parameters
<b>Dictionary - sensor: </b> the sensor object retrieved from the config file<br>
#### Description
a function that establishes a connection to the Arduino and after a successful connection starts to request data based on the interval specified by the user. additionally, this function implements the conversion from raw readings to Amps/Watts. Where 

<b>ADC Voltage </b> = Raw ADC Reading multiplied by VPP Volts Per Point (5.0 / 1024.0 = 0.0048828125) <b>ADCVoltage = ((RawADC*VPP)-OFFSET)</b> <br>
<b>Amps </b> = ADCVoltage devided by Sensor Sinsitivity "a constant defined in the config file as SNS (0.066)" <b> Amps=(ADCVoltage/SNS)</b> <br>
<b>Watts</b> = Amps multiplied by VCC (5V) <b>Watts= Amps*VCC</b>

In [ ]:
def read_serial(sensor):
    try:
        arduino = serial.Serial(sensor['PORT'],settings['BUDRATE'],timeout=settings['TIMEOUT'])
        time.sleep(1)
        flush(arduino)
        while True:
            arduino.write(str.encode('read \n'))
            time.sleep(0.001) 
            unix_ts = str(time.time()).replace('.' , '')
            data = arduino.readline()[:-2].decode("utf-8")
            if data: 
                try:
                    voltage = float(data) * settings['VPP']
                    voltage -= settings['OFFSET']
                    amps = "{:.4f}".format(voltage / settings['SNS'])
                    watts = (voltage / settings['SNS']) * settings['VCC']
                    if(voltage / settings['SNS'] > 10):
                        tmp = "000"
                        payload = str(unix_ts) + "," + sensor['SENSORS_ID'] + "," + tmp + "," +  tmp + "," +tmp+ "," + tmp +"\n"
                    else:
                        payload = str(unix_ts) + "," + sensor['SENSORS_ID'] + "," + data + "," +  "{:.2f}".format(voltage) + "," +str(amps)+ "," + watts +"\n"
                    sstatus = local_save(payload)
                    rstatus = "Disabled"
                    if settings['REMOTE_SAVE'] == True:
                        rstatus = remote_save(payload)
                    if(settings['LOG_LEVEL'] > 0):
                        log(sensor['SENSORS_ID'],payload,rstatus,sstatus)
                except Exception as e:
                    print(FAIL + '[ERROR]' + e)
            flush(arduino)
            time.sleep(settings['POLLING_DELAY'])
    except Exception as e:
        print(WARNING + "[WARNING] attempting to connect to" , sensor['SENSORS_ID'] , e)
        time.sleep(5)
        read_serial(sensor)


## Main
Reads config file and starts multiple threads to handle all sensors defined in the config file.  

In [ ]:
if __name__ == "__main__":
    print(OKGREEN + "[INFO] Starting PowMonDataCollector")
    print(OKGREEN +"[INFO] Loading Settings")
    with open(str(pathlib.Path(__file__).parent.absolute()) + "/settings.yaml", 'r') as stream:
        settings = yaml.safe_load(stream)
    settings['EHOST'] = Fernet(str.encode(settings['LOCAL_DATA_PATH'])).decrypt(str.encode(settings['HOST'])).decode()
    print(OKCYAN +'[Settings]\tFILE_SIZE_LIMIT = ' , settings['FILE_SIZE_LIMIT'] , '\tLOG_LEVEL = ' , settings['LOG_LEVEL'] , '\tSYNC_WRITE =' , settings['SYNC_WRITE'])
    for sensor in settings['SENSORS']:
        print(OKGREEN +"[INFO] Starting thread: " , sensor['SENSORS_ID'])
        x = threading.Thread(target=read_serial, args=(sensor,))
        x.start()
        print(OKGREEN +"[INFO] Started thread: " , sensor['SENSORS_ID'])

# Settings.yaml 
## Config file 
    -BUDRATE: Serial Budrate 
    -EHOST: Not Available
    -FILE_SIZE_LIMIT: Max size of the data file
    -HOST: Not Available
    -LOCAL_DATA_FILE: Name of the local CSV file 
    -LOCAL_DATA_PATH: Not Available
    -LOCAL_DATA_ROOT: Data Root Directory
    -LOG_LEVEL: Log Level
    -OFFSET: Voltage offset, a constant to cancel any voltage generated by the sensor
    -POLLING_DELAY: Data collection delay in seconds
    -PROJECT_ROOT: Root directory
    -REMOTE_SAVE: Enable/Disable Remote Save 
    -SENSORS:
    -- PORT: COM4
    -  SENSORS_ID: VIC
    -- PORT: COM14
    -  SENSORS_ID: ATT
    -SNS: Sensor Sensitivity
    -SYNC_WRITE: Synchronize threads when writing to local file (True or False)
    -TIMEOUT: Arduino connection time out in seconds 
    -VPP: Const Volts Per Point
    -VCC: Const Supply Voltage